In [4]:
from func import *
import numpy as np
import pandas as pd
import itertools

tasks = ['Beer','Fodors-Zagat', 'Walmart-Amazon', 'Amazon-Google', 'DBLP-ACM', 'DBLP-GoogleScholar', 'iTunes-Amazon']
# tasks = ['DBLP-GoogleScholar', 'iTunes-Amazon']
methods = ['SB', 'EQG', 'ESA', 'QG','SA']#,'AE','CTT']

for task in tasks:
    for method in methods:
        
        left_df, right_df,match_df = load_blk_data(task)
        # left_df = left_df.sample(frac= 0.3, replace = False,random_state=42)
        # right_df = right_df.sample(frac= 0.3, replace = False, random_state=42)
        candidates, tot_time, df = trad_blk(task, method, left_df, right_df, match_df, attr_type = 'all')

        col = sens_dict[task][0]

        combinations = list(itertools.product(left_df['id'], right_df['id']))
        combinations_df = pd.DataFrame(combinations, columns=['left_id', 'right_id'])
        combinations_df = combinations_df.rename(columns={'left_id':'id1','right_id':'id2'})
        match_df = match_df.rename(columns={'ltable_id':'id1','rtable_id':'id2'})
        merged_df= pd.merge(combinations_df, df[['id1', 'id2']], 
                            on=['id1', 'id2'], how='left', indicator=True)

        merged_df['pred-label2'] = 0
        merged_df.loc[merged_df['_merge'] == 'both', 'pred-label2'] = 1


        merged_df2= pd.merge(combinations_df, match_df[['id1', 'id2']], 
                            on=['id1', 'id2'], how='left',  indicator=True)

        merged_df2['label'] = 0
        merged_df2.loc[merged_df2['_merge'] == 'both', 'label'] = 1


        combinations = list(itertools.product(left_df[col], right_df[col]))
        combinations_df = pd.DataFrame(combinations, columns=['left_id', 'right_id'])
        print('******')


        a = make_sens_vector(pd.DataFrame({col:combinations_df['left_id']}),task, sens_dict)
        b = make_sens_vector(pd.DataFrame({col:combinations_df['right_id']}),task, sens_dict)
        merged_df2['sens'] = np.logical_or(a,b)
        merged_df['sens'] = np.logical_or(a,b)
        res = merged_df[['pred-label2','sens']]
        res['label'] = merged_df2['label']
        res['pred-label'] = 0
        res['pred-label'] = np.logical_and(res['label']==1, res['pred-label2']==1)


        # Separate the minority and majority groups
        minor = res[res['sens'] == 1]
        major = res[res['sens'] == 0]


        # For Minority group (group 1)
        actual = minor['label']
        predicted = minor['pred-label']
        TP_g1 = np.sum((actual == 1) & (predicted == 1))
        FN_g1 = np.sum((actual == 1) & (predicted == 0))
        FP_g1 = np.sum((actual == 0) & (predicted == 1))
        TN_g1 = np.sum((actual == 0) & (predicted == 0))

        # For Majority group (group 2)
        actual = major['label']
        predicted = major['pred-label']
        TP_g2 = np.sum((actual == 1) & (predicted == 1))
        FN_g2 = np.sum((actual == 1) & (predicted == 0))
        FP_g2 = np.sum((actual == 0) & (predicted == 1))
        TN_g2 = np.sum((actual == 0) & (predicted == 0))


        tpr_g1 = TP_g1 / (TP_g1 + FN_g1)
        fpr_g1 = FP_g1 / (FP_g1 + TN_g1)
        tpr_g2 = TP_g2 / (TP_g2 + FN_g2)
        fpr_g2 = FP_g2 / (FP_g2 + TN_g2)


        EO = np.abs((TP_g1 / (TP_g1+ FN_g1)) - (TP_g2 / (TP_g2+ FN_g2))) + np.abs(FP_g1/ (FP_g1+TN_g1) - FP_g2/ (FP_g2+TN_g2))
        EOP = np.abs(tpr_g1 - tpr_g2)
        DP =  ((TP_g2 + FP_g2)/(TP_g2 + FP_g2 + TN_g2 + FN_g2)) - ((TP_g1 + FP_g1)/(TP_g1 + FP_g1 + TN_g1 + FN_g1))


        metrics = { 'EOP': EOP,'EO': EO,'DP': DP}
        print(task , method)
        for metric, value in metrics.items():
            print(f"{metric}: {value}")
        print('           TP, FN, FP, TN')
        print('minority: ', TP_g1,FN_g1,FP_g1,TN_g1 )
        print('majority: ', TP_g2,FN_g2,FP_g2,TN_g2 )
        print()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


BLAST:   0%|          | 0/20064 [00:00<?, ?it/s]

******
DBLP-GoogleScholar SB
EOP: 0.006479481641468721
EO: 0.006479481641468721
DP: 4.3341896540199086e-08
           TP, FN, FP, TN
minority:  37 0 0 1128018
majority:  460 3 0 14005497



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


BLAST:   0%|          | 0/20064 [00:00<?, ?it/s]

...

Cardinality Edge Pruning:   0%|          | 0/2072 [00:00<?, ?it/s]

******
iTunes-Amazon QG
EOP: 0.0
EO: 0.0
DP: 4.5370072409663817e-07
           TP, FN, FP, TN
minority:  2 0 0 15804497
majority:  11 0 0 18957434



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mohammad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Cardinality Edge Pruning:   0%|          | 0/2072 [00:00<?, ?it/s]

******
iTunes-Amazon SA
EOP: 0.0
EO: 0.0
DP: 4.5370072409663817e-07
           TP, FN, FP, TN
minority:  2 0 0 15804497
majority:  11 0 0 18957434

